In [0]:
%pip install openpyxl
import pandas as pd
import openpyxl
from pyspark.sql import SparkSession
import os
import numpy as np

In [0]:


HEADERS = ['NO_CONTRATO', 'NO_LICITACION', 'GPO', 'GEN', 'ESP', 'DIF', 'VAR', 'CANT_MAX','SOLICITADO', 'PRECIO_NETO', 'RAZON_SOCIAL', 'FECHA_INICIO', 'FECHA_TERMINACION', ]

xlsx_folder = os.path.abspath(os.path.join("..", "market_files" ))

# Conversión a conjunto para comparar la unión interna
needed_headers = set(HEADERS)
missing_summary = {}   # {xlsx_file: [missing_cols...]}
ok_files = []          # files that contain ALL required headers

# Obtener todos los archivos xlsx
xls_files = sorted(
    os.path.join(xlsx_folder, f)
    for f in os.listdir(xlsx_folder)
    if f.lower().endswith(".xlsx")
    )

# Generar un datafame con los encabezados necesarios 
df_bronze = pd.DataFrame()
# Confirmar que los archivos tienen los headers necesarios. 
for xlsx_file in xls_files: 
    pdf = pd.read_excel(
        xlsx_file,
        dtype=str,
        na_filter=False
    )
    file_headers = set(pdf.columns)
    # Interseccion de headdrs
    intersection = needed_headers & file_headers

    # Confirmar que los headers necesarios están presentes. 
    if len(intersection) == len(needed_headers):
        #Agregar la fecha del archivo basado en el prefijo
        file_date_str = os.path.basename(xlsx_file).replace("IMSS_", "").replace(".xlsx", "")
        file_date = pd.to_datetime(file_date_str, format="%Y_%m").replace(day=1)
        headers_out = HEADERS + ['FILE_DATE']
        pdf['FILE_DATE'] = file_date
        # Concatenamos el dataframe
        df_bronze = pd.concat([df_bronze, pdf.loc[:, headers_out]], ignore_index=True)
    else:
        missing = sorted(needed_headers - file_headers)
        missing_summary[xlsx_file] = missing
display(df_bronze.head(10))
#df_bronze.limit(10).display()
# Resumen
print(f"✅ OK los archivos tienen {len(HEADERS)} encabezados presentes): {len(ok_files)}")
if missing_summary:
    print(f"⚠️ Files with missing headers: {len(missing_summary)}")


headers_clean = df_bronze.columns.tolist() 
print(f"✅ Headers extraídos exitosamente: {headers_clean[0:3]}...")

In [0]:
# Retiramos filas con valores ausentes
data_pdf = df_bronze.dropna()
# Creamos el dataframe de spark
spark_df = spark.createDataFrame(data_pdf) \
                 .toDF(*headers_clean)

# Guardamos la tabla Delta
table = "workspace.default.bronze_market_data"
spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(table)

print(f"✅ Tabla e ingestión completada: {table}")